# Storyden Boot Time Analysis

This notebook analyzes boot time logs to identify performance bottlenecks.

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Configure pandas to show full text in cells
pd.set_option('display.max_colwidth', None)  # No truncation
pd.set_option('display.max_rows', 100)       # Show up to 100 rows
pd.set_option('display.width', None)         # Auto-detect width

sns.set_theme(style="whitegrid")
%matplotlib inline

In [2]:
# Load the JSONL logs with error handling
logs = []
errors = 0
with open('logs.jsonl', 'r') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            logs.append(json.loads(line))
        except json.JSONDecodeError as e:
            errors += 1
            print(f"Warning: Skipping invalid JSON on line {line_num}: {e}")
            print(f"  Line content: {line[:100]}...")

if errors > 0:
    print(f"\nSkipped {errors} invalid lines")

df = pd.DataFrame(logs)
print(f"Total log entries: {len(df)}")
df.head()

  Line content: c^C{"time":"2025-10-19T21:39:04.158627+01:00","level":"DEBUG","msg":"Closing subscriber, waiting for...

Skipped 1 invalid lines
Total log entries: 638


,time,level,msg,component,constructor,types,elapsed,duration,elapsed_ms,duration_ms,...,subscriber_name,boot_time,address,api_address,web_address,log_level,total_startup_time,total_startup_ms,pubsub_uuid,timeout
0,2025-10-19T21:39:02.35745+01:00,INFO,dependency constructed,bootstrap,go.uber.org/fx.New.func1(),fx.Lifecycle,2667.0,2667.0,0.002,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-10-19T21:39:02.357529+01:00,INFO,dependency constructed,bootstrap,go.uber.org/fx.(*App).shutdowner-fm(),fx.Shutdowner,111000.0,108333.0,0.111,0.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-10-19T21:39:02.357533+01:00,INFO,dependency constructed,bootstrap,go.uber.org/fx.(*App).dotGraph-fm(),fx.DotGraph,115083.0,4083.0,0.115,0.004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-10-19T21:39:02.357535+01:00,INFO,dependency constructed,bootstrap,main.Start.func2(),context.Context,117583.0,2500.0,0.117,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-10-19T21:39:02.357561+01:00,INFO,dependency constructed,bootstrap,github.com/Southclaws/storyden/internal/config...,config.Config,143417.0,25834.0,0.143,0.025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Filter for dependency construction events
deps = df[df['msg'] == 'dependency constructed'].copy()

# Use the *_ms fields if available, otherwise fall back to converting from nanoseconds
if 'elapsed_ms' in deps.columns:
    deps['elapsed_ms'] = deps['elapsed_ms']
else:
    deps['elapsed_ms'] = deps['elapsed'] / 1000

if 'duration_ms' in deps.columns:
    deps['duration_ms'] = deps['duration_ms'].fillna(0)
else:
    deps['duration_ms'] = deps['duration'].fillna(0) / 1000

print(f"Total dependencies constructed: {len(deps)}")
print(f"\nDependencies with timing info: {(deps['duration_ms'] > 0).sum()}")

Total dependencies constructed: 187

Dependencies with timing info: 187


## Slowest Constructors (by duration)

In [4]:
# Top 20 slowest constructors
slowest = deps[deps['duration_ms'] > 0].nlargest(20, 'duration_ms')[['constructor', 'duration_ms', 'elapsed_ms']]
slowest

,constructor,duration_ms,elapsed_ms
1,go.uber.org/fx.(*App).shutdowner-fm(),0.108,0.111
59,github.com/Southclaws/storyden/app/resources/l...,0.066,0.506
30,github.com/Southclaws/storyden/app/resources/a...,0.063,0.341
18,github.com/Southclaws/storyden/internal/infras...,0.041,0.243
8,github.com/Southclaws/storyden/internal/infras...,0.028,0.179
4,github.com/Southclaws/storyden/internal/config...,0.025,0.143
157,github.com/Southclaws/storyden/app/transports/...,0.025,0.895
66,github.com/Southclaws/storyden/app/resources/p...,0.023,0.542
111,github.com/Southclaws/storyden/app/services/as...,0.022,0.732
144,github.com/Southclaws/storyden/internal/infras...,0.022,0.847


In [ ]:
# Visualize slowest constructors
# Lower threshold since constructor durations are approximate and often small
top_slow = deps[deps['duration_ms'] > 0.1].nlargest(15, 'duration_ms')

if len(top_slow) > 0:
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top_slow)), top_slow['duration_ms'])
    plt.yticks(range(len(top_slow)), [c.split('/')[-1][:50] for c in top_slow['constructor']], fontsize=9)
    plt.xlabel('Duration (ms)')
    plt.title(f'Top {len(top_slow)} Slowest Constructors (by approximate duration)')
    plt.tight_layout()
    plt.show()
    
    print(f"\nNote: Constructor durations shown: {len(top_slow)}")
    print(f"Total constructors: {len(deps)}")
    print(f"Constructors with duration > 0.1ms: {(deps['duration_ms'] > 0.1).sum()}")
else:
    print("No constructors with duration > 0.1ms found")
    print(f"\nConstructor duration range: {deps['duration_ms'].min():.4f}ms to {deps['duration_ms'].max():.4f}ms")
    print(f"Mean constructor duration: {deps['duration_ms'].mean():.4f}ms")

## Function Invocations

In [ ]:
# Function invocations
invocations = df[df['msg'] == 'function invoked'].copy()

# Use the *_ms field if available
if 'duration_ms' in invocations.columns:
    invocations['duration_ms'] = invocations['duration_ms']
else:
    invocations['duration_ms'] = invocations['duration'] / 1000

slowest_invocations = invocations.nlargest(15, 'duration_ms')[['function', 'duration_ms']]
slowest_invocations

In [ ]:
# Visualize slowest invocations
plt.figure(figsize=(12, 8))
plt.barh(range(len(slowest_invocations)), slowest_invocations['duration_ms'])
plt.yticks(range(len(slowest_invocations)), [f.split('/')[-1][:60] for f in slowest_invocations['function']])
plt.xlabel('Duration (ms)')
plt.title('Top 15 Slowest Function Invocations')
plt.tight_layout()
plt.show()

## OnStart Lifecycle Hooks

In [ ]:
# OnStart hooks
hooks = df[(df['msg'] == 'lifecycle hook completed') & (df['hook'] == 'OnStart')].copy()

# Use the *_ms field if available
if 'duration_ms' in hooks.columns:
    hooks['duration_ms'] = hooks['duration_ms']
else:
    hooks['duration_ms'] = hooks['duration'] / 1000

slowest_hooks = hooks.nlargest(15, 'duration_ms')[['callee', 'function', 'duration_ms']]
slowest_hooks

In [ ]:
# Visualize slowest hooks
plt.figure(figsize=(12, 8))
plt.barh(range(len(slowest_hooks)), slowest_hooks['duration_ms'])
plt.yticks(range(len(slowest_hooks)), [c.split('/')[-1][:50] for c in slowest_hooks['callee']])
plt.xlabel('Duration (ms)')
plt.title('Top 15 Slowest OnStart Hooks')
plt.tight_layout()
plt.show()

## Boot Timeline

In [ ]:
# Total boot time
total = df[df['msg'] == 'application started successfully']
if len(total) > 0:
    # Use total_startup_ms if available
    if 'total_startup_ms' in total.columns and pd.notna(total.iloc[0]['total_startup_ms']):
        total_time_ms = total.iloc[0]['total_startup_ms']
    else:
        total_time_ms = total.iloc[0]['total_startup_time'] / 1000000
    print(f"Total Boot Time: {total_time_ms:.2f}ms ({total_time_ms/1000:.2f}s)")
else:
    print("Total boot time not found in logs")

In [ ]:
# Timeline of major events
timeline_data = []

# Add invocations
for _, row in invocations.iterrows():
    timeline_data.append({
        'name': row['function'].split('/')[-1][:40],
        'duration': row['duration_ms'],
        'type': 'Invocation'
    })

# Add OnStart hooks
for _, row in hooks.iterrows():
    timeline_data.append({
        'name': row['callee'].split('/')[-1][:40],
        'duration': row['duration_ms'],
        'type': 'OnStart Hook'
    })

timeline_df = pd.DataFrame(timeline_data)

# Show only items > 10ms
significant = timeline_df[timeline_df['duration'] > 10].sort_values('duration', ascending=False).head(20)

if len(significant) > 0:
    plt.figure(figsize=(14, 10))
    colors = {'Invocation': 'steelblue', 'OnStart Hook': 'coral'}
    
    for i, (idx, row) in enumerate(significant.iterrows()):
        plt.barh(i, row['duration'], color=colors.get(row['type'], 'gray'))
    
    plt.yticks(range(len(significant)), significant['name'])
    plt.xlabel('Duration (ms)')
    plt.title('Boot Timeline - Significant Events (>10ms)')
    plt.legend(handles=[plt.Rectangle((0,0),1,1, color=c) for c in colors.values()], 
               labels=colors.keys())
    plt.tight_layout()
    plt.show()

## Summary Statistics

In [ ]:
print("=== BOOT TIME BREAKDOWN ===")
print(f"\nTotal Boot Time: {total_time_ms:.2f}s\n")

print("Top 3 Bottlenecks:")
print("------------------")
for i, (_, row) in enumerate(slowest_invocations.head(3).iterrows(), 1):
    pct = (row['duration_ms'] / total_time_ms) * 100
    print(f"{i}. {row['function'].split('/')[-1][:60]}")
    print(f"   {row['duration_ms']:.2f}ms ({pct:.1f}% of total)\n")

print("\nOnStart Hook Summary:")
print("--------------------")
total_hook_time = hooks['duration_ms'].sum()
print(f"Total OnStart hook time: {total_hook_time:.2f}ms")
print(f"Percentage of boot time: {(total_hook_time/total_time_ms)*100:.1f}%")

print("\nInvocation Summary:")
print("------------------")
total_invocation_time = invocations['duration_ms'].sum()
print(f"Total invocation time: {total_invocation_time:.2f}ms")
print(f"Percentage of boot time: {(total_invocation_time/total_time_ms)*100:.1f}%")